# Iterators

Objects can be an iterable, an iterator or both. According to PEP234

1. An object can be iterated over with "for" if it implements
   `__iter__()` or `__getitem__()`.

2. An object can function as an iterator if it implements `next()`.

`all()`: returns `True` if all elements of an iterable are `True`...

In [1]:
print("all([True, True, True, True]) =", all([True, True, True, True]))
print("all([True, True, False, True]) =", all([True, True, False, True]))

all([True, True, True, True]) = True
all([True, True, False, True]) = False


`any()`: returns True if all elements of an iterable are `True`...

In [2]:
print("any((False, False, False, True)) =", any((False, False, False, True)))
print("any((False, False, False, False)) =", any((False, False, False, False)))

any((False, False, False, True)) = True
any((False, False, False, False)) = False


`min/max()`: return min/max of an iterable...

In [2]:
print("max(['Bob', 'Tim', 'Andy']) =", max(['Bob', 'Tim', 'Andy']))
print("min([1, 2, -3]) =", min([1, 2, -3]))

max(['Bob', 'Tim', 'Andy']) = Tim
min([1, 2, -3]) = -3


Use `next()` function to read lines from file...

In [ ]:
with open('7_iterators.ipynb', 'r') as f:        # f is a file handle but also an iterator...
    try:                                         # .. it has a __next__ attribute
        while True:
            current = next(f)
            print(current.strip())
    except StopIteration:                        # next() will throw StopIteration exception when done
        print('EOF!')

Common way to use an iterator is in a `for` loop...

In [6]:
my_list = [1, 5, 3, 'Bob', 2]    # my_list is iterable
for an_item in my_list:
    print(an_item)

1
5
3
Bob
2


A `list` is iterable but it is not an iterator itself. You can get an iterator from a `list` using the `iter()` function. `list` has an `__iter__` attribute which will return an iterator but no `__next__` which would make it an iterator.

In [7]:
my_iter = iter(my_list)
for an_item in my_iter:
    print(an_item)

1
5
3
Bob
2


You can use `iter()` and `next()` to implement a for loop. Note this is how a `for` loop is actually implemented.

In [8]:
my_iter = iter(my_list)
while True:
    try:
        # next() will throw StopIteration if nothing left in iterable
        print(next(my_iter))
    except StopIteration:
        print("StopIteration exception raised so must be at end of list!")
        break

1
5
3
Bob
2
StopIteration exception raised so must be at end of list!


Alternatively, you can use the `__next__()` attribute of the iterator...

In [10]:
my_iter = iter(my_list)
while True:
    try:
        # next() will throw StopIteration if nothing left in iterable
        print(my_iter.__next__())
    except StopIteration:
        print("End of list!")
        break

1
5
3
Bob
2
End of list!


We can create an subscriptable class by implementing the `__getitem__` attribute...

In [27]:
class GetItemClass:
    def __getitem__(self, item):        # item can be an integer or a slice object
        if type(item) is int:
            return 2**item
        elif type(item) is slice:
            return [2**x for x in range(item.start, item.stop, item.step)]

my_getitem_instance = GetItemClass()
print("my_getitem_instance[0] =", my_getitem_instance[0])
print("my_getitem_instance[1] =", my_getitem_instance[1])
print("my_getitem_instance[2] =", my_getitem_instance[2])
print("Because GetItemClass is subscriptable it is also iterable")
for my_value in my_getitem_instance:
    print("my_value =", my_value)
    if my_value > 32:          # because GetItemClass is an infinitely iterable need to break
        break

my_getitem_instance[0] = 1
my_getitem_instance[1] = 2
my_getitem_instance[2] = 4
Because GetItemClass is subscriptable it is also iterable
my_value = 1
my_value = 2
my_value = 4
my_value = 8
my_value = 16
my_value = 32
my_value = 64


We can also use slicing with `GetItemClass` as it supports `__getitem__`

In [10]:
for my_value in my_getitem_instance[4:12:2]:
    print("my_value =", my_value)

in GetItemClass.__getitem__
my_value = 16
my_value = 64
my_value = 256
my_value = 1024


 `iter()` will return an iterator even though `__iter__` is not explicitly defined in class definition. `iter()` will return the base `object` base class Iterator object. `next()` also works on this iterator by calling `__getitem__` with increasing `item` values.

In [14]:
my_getitem_iterator = iter(my_getitem_instance)
print("my_getitem_iterator =", my_getitem_iterator)
print("dir(my_getitem_iterator) =", dir(my_getitem_iterator))
print("next(my_getitem_iterator) =", next(my_getitem_iterator))
print("next(my_getitem_iterator) =", next(my_getitem_iterator))

my_getitem_iterator = <iterator object at 0x0000017068F55180>
dir(my_getitem_iterator) = ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']
next(my_getitem_iterator) = 1
next(my_getitem_iterator) = 2


We can create an iterable class that is also an iterator by implementing `__iter__` and `__next__` attributes...

In [34]:
class IterableClass:
    """class is iterable but also an iterator"""        # note this is not always true - sometimes an iterable...
    def __init__(self, limit):                          # ...will return a separate iterator instance e.g. a list
        self.limit = limit

    def __iter__(self):                 # called when you use iter() on an instance of IterableClass
        # need to reset each time iter() is called else only works once
        self.value = 1
        return self                     # returns itself as the iterator

    def __next__(self):                 # as Iterable class is also an iterator we need to implement __next__
        if self.value < self.limit:
            self.value *= 2
            return self.value
        else:
            raise StopIteration         # iteration will cease when this is raised

Use it as an iterable object

In [36]:
iterable_object = IterableClass(8)
for value in iterable_object:
    print(value)

2
4
8


Or use the iterator

In [37]:
iterator_object = iter(IterableClass(8))
while True:
    try:
        print(next(iterator_object))
    except StopIteration:
        break

2
4
8


 Note you can pass `iter()` a function and a sentinel, a value that raises the `StopIteration` exception if returned from function...

In [23]:
import random

def my_random_number_generator():
    return random.randint(1, 10)

sentinel_based_iterator = iter(my_random_number_generator, 5)   # 5 is the sentinel
while True:
    try:
        print(next(sentinel_based_iterator))
    except StopIteration:
        print("Jackpot! my_random_number_generator() function has returned 5!")
        break

1
3
Jackpot! my_random_number_generator() function has returned 5!


You can also use it with `iter()` in a for loop...

In [14]:
for random_value in iter(my_random_number_generator, 5):
    print(random_value)

6
10
8
6


You can use `zip()` to create an iterable of tuples that aggregate from a number of iterables...

In [24]:
my_list_a = ['Bob', 'Susan', 'John', 'Lucy']
my_list_b = [1, 2, 3]
my_list_c = [3.1, 3.4, 1.4, 8.3]

# will terminate on shortest of the iterables
for zipped_tuple in zip(my_list_a, my_list_b, my_list_c):
    # gets the nth item from each iterable in the zip list and puts them in a tuple
    print(zipped_tuple)

('Bob', 1, 3.1)
('Susan', 2, 3.4)
('John', 3, 1.4)


We can create an iterable class that returns a separate iterator object. Note this is useful if you want multiple separate iterators looping over the same iterable object.

In [40]:
class StudentIteratorClass:
    def __init__(self, student_iterable):
        print("Iterator: initialise")
        self.student_iterable = student_iterable
        self.index = 0

    def __next__(self):                                     # as an iterator we need to implement __next__
        if self.index < len(self.student_iterable.students):
            _next_val = self.student_iterable.students[self.index]
            self.index += 1
            return _next_val
        else:
            raise StopIteration

class StudentIterableClass:
    def __init__(self):
        print("Iterable: initialise")
        self.students = ['Bob', 'Sally', 'Jake']

    def __iter__(self):                             # we want to return our iterator rather than base class
        print("Iterable: get iterator")
        return StudentIteratorClass(self)           # pass the iterable to the iterator

So we can iterate over the iterable object as it implements `__iter__` but it is not subscriptable.

In [41]:
my_instance = StudentIterableClass()
try:
    print(my_instance[0])
except TypeError:
    print("Iterable but not subscriptable!")
for student in my_instance:
    print("Student =", student)

Iterable: initialise
Iterable but not subscriptable!
Iterable: get iterator
Iterator: initialise
Student = Bob
Student = Sally
Student = Jake


Or we can use one or more iterators derived from the same iterable object

In [43]:

my_instance = StudentIterableClass()
my_student_iterator1 = iter(my_instance)
my_student_iterator2 = iter(my_instance)
print("type(my_instance_iterator) =", type(my_student_iterator1))
try:
    for student in my_student_iterator1:
        print(student)
except TypeError:
    print("Iterator not iterable!")
while True:
    try:
        print("Student =", next(my_student_iterator1))
        print("Student =", next(my_student_iterator2))
    except StopIteration:
        break

Iterable: initialise
Iterable: get iterator
Iterator: initialise
Iterable: get iterator
Iterator: initialise
type(my_instance_iterator) = <class '__main__.StudentIteratorClass'>
Iterator not iterable!
Student = Bob
Student = Bob
Student = Sally
Student = Sally
Student = Jake
Student = Jake
